<a href="https://colab.research.google.com/github/prateekchandrajha/worldquant-online-algorithms/blob/main/Follow_The_Leader_Algorithms_FTRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math
import numpy as np

random.seed(1)

References for FTRL Algorithm:

- "Follow the Regularized Leader" method provides a framework to design and analyze online algorithms in a versatile and well-motivated way. We will be using FTRL to simulate trading before the final submission in February.

- https://medium.com/@dhirajreddy13/factorization-machines-and-follow-the-regression-leader-for-dummies-7657652dce69

- FTRL and Mirror Descent Algorithms: https://storage.googleapis.com/pub-tools-public-publication-data/pdf/37013.pdf

In [2]:
class FollowTheRegularizedLeader(object):
    def __init__(self, alpha, beta, l1, l2, features, reg_func, indices):
        self.alpha = alpha
        self.beta = beta
        self.l1 = l1
        self.l2 = l2
        self.reg_func = reg_func
        self.indices = [0]
        self.count = 0
        for index in indices:
            self.indices.append(index)
        self.sum_of_gradients_squared = [0.] * features
        self.weights = {}
        self.temp_weights = []
        i = 0
        while i < features:
            self.temp_weights.append(random.random())
            i += 1

    def update(self, probability, result):
        gradient = probability - result
        gradient_squared = math.pow(gradient, 2)
        self.count += 1
        if self.reg_func is "RDA":
            for i in self.indices:
                sigma = (math.sqrt(self.sum_of_gradients_squared[i] + gradient_squared)) / (self.alpha * self.count)
                self.temp_weights[i] += -(sigma * self.weights[i]) + gradient
                self.sum_of_gradients_squared[i] += gradient_squared
        elif self.reg_func is "OPG":
            for i in self.indices:
                sigma = (math.sqrt(self.sum_of_gradients_squared[i] + gradient_squared) - math.sqrt(self.sum_of_gradients_squared[i])) / self.alpha
                self.temp_weights[i] += -(sigma * self.weights[i]) + gradient
                self.sum_of_gradients_squared[i] += gradient_squared

    def predict(self):
        weights = {}
        function = "Sigmoid"
        w_inner_x = float(0)
        for i in self.indices:
            sign = float(np.sign(self.temp_weights))
            if sign * self.temp_weights[i] <= self.l1:
                weights[i] = float(0)
            else:
                weights[i] = (sign * self.l1 - self.temp_weights[i]) / ((self.beta + math.sqrt(self.sum_of_gradients_squared[i])) / self.alpha + self.l2)
            w_inner_x += weights[i]

        self.weights = weights

        if function is "Sigmoid":
            probability = float(1) / (float(1) + math.exp(-max(min(float(w_inner_x), float(100)), float(-100))))
        elif function is "ReLU":
            probability = max(float(0), max(min(float(w_inner_x), float(100)), float(-100)))

        return probability

    @staticmethod
    def log_loss(true_label, predicted, eps=1e-15):
        p = np.clip(predicted, eps, 1 - eps)
        if true_label > 0:
            return -math.log(p)
        else:
            return -math.log(1 - p)

In [ ]:
# Declaring FTRL Parameters

alpha = 0.2
beta = 0.5
l1 = 0.32
l2 = 0.32
# features = number of features
# reg_func = either "RDA" or "OPG"
# indices

FTRL_simulate = FollowTheRegularizedLeader(alpha, beta, l1, l2, features, reg_func, indices) # declaring an object of FTRL algorithm class for performing simulations